In [2]:
import numpy as np
import xgc
import math
#import xgcjrm as xgc
from matplotlib.tri import Triangulation, LinearTriInterpolator
import matplotlib.pyplot as plt
from scipy.interpolate import splrep, splev


#limit data to the [Rmin,Rmax,Zmin,Zmax] box, and read only the first two toroidal planes
Rmin=None#2.2
Rmax=None#2.31
Zmin=None#-0.25
Zmax=None#0.4
phi_start=0
phi_end=1

#fileDirec='/scratch1/scratchdirs/giannos/particle_pinch/particle_pinch_idl/hdf5'
fileDir= '/cygdrive/d/particle_pinch_idl/hdf5'

def getMeshAndCuts(fileDir,nRi,nZi,Rmin=None,Rmax=None,Zmin=None,Zmax=None,phi_start, phi_end):

	global loader,ne,pot,psin,RZ,tri,time,psin1d,psin001d,Te1d,ne1d,pot001d,bfield,pot0,dpot,dpot,pot,Tigc1d,nigc1d
	global Nplanes,Ntimes,dt
	global Ri,Zi,RI,ZI,triObj
	global sepInds,psinvalues,jcut,psii,Zcut
	
	loader=xgc.load(fileDir,Rmin=Rmin,Rmax=Rmax,Zmin=Zmin,Zmax=Zmax,phi_start=phi_start,phi_end=phi_end)

	ne = loader.calcNeTotal()
	pot = loader.calcPotential()
	print('Rmin = ',loader.Rmin)
	print('Rmax = ',loader.Rmax)
	print('Zmin = ',loader.Zmin)
	print('Zmax = ',loader.Zmax)
	print('ne.shape = ',ne.shape)
	
	# (could have imported * from xgc but this way we see explicitly what we are getting)

	# arrays
	psin = loader.psin
	RZ = loader.RZ
	tri = loader.tri
	time = loader.time
	psin1d = loader.psin1d
	psin001d = loader.psin001d
	Te1d = loader.Te1d
	ne1d = loader.ne1d
	pot001d = loader.pot001d
	#bfield = loader.bfield
	pot0 = loader.pot0
	dpot = loader.dpot
	pot = loader.pot
	Tigc1d = loader.Ti1d
	nigc1d = loader.ni1d

	# scalars
	Nplanes = loader.Nplanes
	Ntimes = loader.Ntimes
	dt = loader.dt

	#setup mesh grid
	if nRi == None:
		nRi = 100
	if nZi == None:
		nZi = 100
	Ri = np.linspace(loader.Rmin,loader.Rmax,nRi)
	Zi = np.linspace(loader.Zmin,loader.Zmax,nZi)
	(RI,ZI) = np.meshgrid(Ri,Zi)
	# set up to interpolate using the TriInterpolator class. Should be what tricontourf() uses
	triObj = Triangulation(RZ[:,0],RZ[:,1],tri)

	# get indices of RZ vertices on the separatrix
	sepInds = np.where(np.abs(loader.psin-1.0)<1e-4)[0]

	# flux surface values on the triangular mesh
	psinvalues = np.sort(np.array(list(set(np.round(psin,4))))) # 4 round to 4 decimal places; adjust?

	# locate j value where Z = Zcut
	if Zcut == None:
		Zcut = Zi.mean()
	jcut = int(round(np.mean(np.where(np.abs(Zi-Zcut)<1.e-2))))
	#print "Zi[0:5] = ",Zi[0:5]
	#print "Zi[-5:0] = ",Zi[-5:-1]
	#print "Zcut = ",Zcut
	#print "jcut = ",jcut


	# calculate psi(Ri) = psii at the cut
	tci=LinearTriInterpolator(triObj,psin)
	psiout=tci(RI,ZI)
	psii = psiout[jcut,:] # a 1D array of psin values at each Ri along the cut




def getcutvalue(arrRZ,j=None):
	"""Calculates the values at Ri along Zcut of arrRZ where arr is a mesh array
	usage:
		netmpi = getcutvalue(ne[:,iz,it]) # where iz is the toroidal plane, and it the time index
	"""
	global triObj,RI,ZI,arrout,tci,jcut,RI,ZI
	# the following construction allows the def to occur before jcut takes a value
	if j==None:
		j = jcut 
	tci=LinearTriInterpolator(triObj,arrRZ)
	arrout=tci(RI,ZI)
	return(arrout[j,:])


#so far the same as the example for calculating potential and n_e and interpolating
def getcutvRvZ(iz,it)
    global Zi,Ri,pot,jcut,bfield,B2
    dZ = Zi[1]-Zi[0]
    dR = Ri[1]-Ri[0]
    theta = (2*math.pi)/Nplanes
    #Loading B-field 
    bfield=loader.loadBfield()
    #calculate values at the cut
    br = getcutvalue(bfield[:,0])
    bz = getcutvalue(bfield[:,1])
    bzeta = getcutvalue(bfield[:,2])
    BfMag=[]
    for i in range(len(br)):
        BfMag.append(np.sqrt(br[i]*br[i] + bz[i]*bz[i] + bzeta[i]*bzeta[i]))
#Calculating the potential gradient
# dphi/dZ    
    potim1 = getcutvalue(pot[:,iz,it],jcut-1)
    poti = getcutvalue(pot[:,iz,it],jcut)
    potip1 = getcutvalue(pot[:,iz,it],jcut+1)
    dphidZ = (potip1-potim1)/(2.*dZ)
# dphi/dR
    dphidR=[]
    length = len(getcutvalue(pot[:,iz,it],jcut))
    iterlist = list(range(0,length))
    for i in iterlist[1:]:
        potrm1 = getcutvalue(pot[:,iz,it],jcut)[i-1]
        potrp1 = getcutvalue(pot[:,iz,it],jcut)[i+1]
        dphidR.append((potrp1 - potrm1)/(2.*dR))
#dphi/dzeta
    potzp1 = getcutvalue(pot[:,iz+1,it],jcut) 
    potz = getcutvalue(pot[:,iz,it],jcut)
    dphidzeta=[]
    for i in len(Ri):
        dphidzeta.append = (potzp1[i] - potz[i])/(Ri[i]*theta)
#Calculating the ExB drift
    vR = []
    for i in len(dphidZ):
        vR.append(dphidZ[i]*bzeta[i] - dphidzeta[i]*bz[i])
    vZ = []
    for i in iterlist[1:]:
        vZ.append(br[i]*dphidzeta[i] - dphidR[i]*bzeta[i])
    return(vR,vZ)

neNorm=np.einsum('i...j,i...->i...j',n_e,1./n_e[:,:,0])
getMeshAndCuts(fileDirec)
VR, VZ = getcutvRvZ(0,80)
plt.plot(neNorm,VR)
plt.show

#Plotting

SyntaxError: invalid syntax (<ipython-input-2-8453b277c2ef>, line 112)